In [ ]:
# Multi CLass Classification usnf SVM Linear

# Step 1: Read the dataset and select The required features (Category0 and Questions)
# Step 2: Encoding the target feature items in Category0
#         {'ABBREVIATION', 'DESCRIPTION', 'ENTITY', 'HUMAN', 'LOCATION', 'NUMERIC'} to
#         {0,1,2,3,4,5}
# Step 3: Split the data set into Train and Test
# Step 4: Vectorize the Input Data (Questions) using TFIDF also using N Gram(1,2)
# Step 5: Algorithm : SVM Linear
# Step 6: Vectorize the user input using trained tfIDF model and Pass interger convered matric to model for prediction

In [3]:
import numpy as np
import pandas as pd


data = pd.read_csv("C:\MyWork\MyLearning\Career Growth\ML\Files\DataSet\Question_Classification_Dataset.csv")

In [31]:
data.shape

(5452, 3)

In [4]:
data.head(3)

,Unnamed: 0,Questions,Category0,Category1,Category2
0,0,How did serfdom develop in and then leave Russ...,DESCRIPTION,DESC,manner
1,1,What films featured the character Popeye Doyle ?,ENTITY,ENTY,cremat
2,2,How can I find a list of celebrities ' real na...,DESCRIPTION,DESC,manner


In [6]:
# Condsider 'Questions' and 'Category0'

data  = data[['Category0','Questions']]

data.head()

,Category0,Questions
0,DESCRIPTION,How did serfdom develop in and then leave Russ...
1,ENTITY,What films featured the character Popeye Doyle ?
2,DESCRIPTION,How can I find a list of celebrities ' real na...
3,ENTITY,What fowl grabs the spotlight after the Chines...
4,ABBREVIATION,What is the full form of .com ?


In [20]:
set(data['Category0'])

{'ABBREVIATION', 'DESCRIPTION', 'ENTITY', 'HUMAN', 'LOCATION', 'NUMERIC'}

In [21]:
# Distinct elements in category0
import matplotlib.pyplot as plt

data['Category0'].value_counts()

ENTITY          1250
HUMAN           1223
DESCRIPTION     1162
NUMERIC          896
LOCATION         835
ABBREVIATION      86
Name: Category0, dtype: int64

In [23]:
# Convert target varibale values to Integer

data['Category0_vector'] = data['Category0'].map({
                                                    'DESCRIPTION' : 0,
                                                    'ENTITY' : 1,
                                                    'ABBREVIATION' : 2,
                                                    'HUMAN' : 3,
                                                    'NUMERIC' : 4,
                                                    'LOCATION' : 5
                                             })

In [25]:
data.head(3)

,Category0,Questions,Category0_vector
0,DESCRIPTION,How did serfdom develop in and then leave Russ...,0
1,ENTITY,What films featured the character Popeye Doyle ?,1
2,DESCRIPTION,How can I find a list of celebrities ' real na...,0


In [61]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(data['Questions'],data['Category0_vector'],test_size=0.2,random_state=0)

In [63]:
X_train.shape

(4361,)

In [64]:
X_train[:4]

4644    Where was the largest concentration camp in Wo...
599     In What city or state do the most gay men live...
2315    How many American soldiers have died for their...
1095    What are the only two states that incorporate ...
Name: Questions, dtype: object

In [66]:
y_train[:4]

4644    5
599     5
2315    4
1095    5
Name: Category0_vector, dtype: int64

In [67]:
# Extract the features using TFIDF alsong with N Gram (Bi Gram)

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range = (1,2)).fit(data['Questions'])

X_train_vect = vect.transform(X_train)
X_test_vect = vect.transform(X_test)

In [68]:
X_train_vect.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [69]:
X_train_vect.todense().shape

(4361, 32705)

In [96]:
# Train the model using Linear SVC

from sklearn.svm import SVC
model = SVC(kernel='linear',probability=True)

model.fit(X_train_vect, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [105]:
category = {0: "DESCRIPTION",1:"ENTITY",2:"ABBREVIATION",3:"HUMAN",4:"NUMERIC",5:"LOCATION"}

def vectorizeSenetnce(text):
    
    # Transform the text to vector(TF IDF)
    vector = vect.transform([text])
    
    return vector


def predictCategory(model,question):
    
    # get the vectorized senetnce
    vector = vectorizeSenetnce(question)
    
    value = model.predict(vector)[0]
    
    # return the respected category
    return category[value]
    

In [106]:
predictCategory(model,'How did serfdom develop in and then leave Russia ?')

'DESCRIPTION'

In [124]:
# choose a random element from a list
from random import seed
from random import choice
# seed random number generator
seed(1)
# prepare a sequence
sequence = [i for i in range(20)]
# make choices from the sequence

index = []

for _ in range(10):
    selection = choice(sequence)
    index.append(selection)
    
actual_output = []
predicted_output = []

df = data.iloc[index,:]

In [125]:
df = df[['Category0','Questions']]

In [126]:
df

,Category0,Questions
4,ABBREVIATION,What is the full form of .com ?
18,DESCRIPTION,What is `` Nine Inch Nails '' ?
2,DESCRIPTION,How can I find a list of celebrities ' real na...
8,DESCRIPTION,What are liver enzymes ?
3,ENTITY,What fowl grabs the spotlight after the Chines...
15,LOCATION,What sprawling U.S. state boasts the most airp...
14,ENTITY,What is considered the costliest disaster the ...
15,LOCATION,What sprawling U.S. state boasts the most airp...
12,HUMAN,Who was The Pride of the Yankees ?
6,HUMAN,What team did baseball 's St. Louis Browns bec...


In [132]:
for index, row in df.iterrows():
    
    actual_output.append(row['Category0'])
    
    issue = row['Questions']
    
    predicted_output.append(predictCategory(model,issue))

In [133]:
pd.DataFrame({"Actual":actual_output,"Predicted":predicted_output})

,Actual,Predicted
0,ABBREVIATION,ENTITY
1,DESCRIPTION,DESCRIPTION
2,DESCRIPTION,DESCRIPTION
3,DESCRIPTION,DESCRIPTION
4,ENTITY,ENTITY
5,LOCATION,LOCATION
6,ENTITY,ENTITY
7,LOCATION,LOCATION
8,HUMAN,HUMAN
9,HUMAN,HUMAN
